In [186]:
import pandas as pd
from  sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [163]:
df = pd.read_csv('C:\\Users\\ACER\\OneDrive - Universitas Airlangga\\Kuliah\\Learn-MLZoomCamp2023\\Learn-MLZoomCamp2023\\Project\\Midterm - Project\\Balaji Fast Food Sales.csv')
df.head()

,order_id,date,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,received_by,time_of_sale
0,1,07-03-2022,Aalopuri,Fastfood,20,13,260,NaN,Mr.,Night
1,2,8/23/2022,Vadapav,Fastfood,20,15,300,Cash,Mr.,Afternoon
2,3,11/20/2022,Vadapav,Fastfood,20,1,20,Cash,Mr.,Afternoon
3,4,02-03-2023,Sugarcane juice,Beverages,25,6,150,Online,Mr.,Night
4,5,10-02-2022,Sugarcane juice,Beverages,25,8,200,Online,Mr.,Evening


In [164]:
#drop some unuseful feature
del df['order_id']
del df['date']

In [165]:
df.columns

Index(['item_name', 'item_type', 'item_price', 'quantity',
       'transaction_amount', 'transaction_type', 'received_by',
       'time_of_sale'],
      dtype='object')

In [166]:
df.shape

(1000, 8)

In [167]:
df.isnull().sum()

item_name               0
item_type               0
item_price              0
quantity                0
transaction_amount      0
transaction_type      107
received_by             0
time_of_sale            0
dtype: int64

In [168]:
df.dropna(subset=['transaction_type'], inplace=True)

In [169]:
df.isnull().sum()

item_name             0
item_type             0
item_price            0
quantity              0
transaction_amount    0
transaction_type      0
received_by           0
time_of_sale          0
dtype: int64

In [170]:
df.describe()

,item_price,quantity,transaction_amount
count,893.000000,893.000000,893.000000
mean,33.359462,8.100784,272.603583
std,15.000808,4.390283,202.457635
min,20.000000,1.000000,20.000000
25%,20.000000,4.000000,120.000000
50%,25.000000,8.000000,240.000000
75%,50.000000,12.000000,360.000000
max,60.000000,15.000000,900.000000


In [171]:
columns = list(df.columns)

for i in columns :
    print(i)
    print(df[i].nunique())
    print(df[i].unique())
    print("")

item_name
7
['Vadapav' 'Sugarcane juice' 'Panipuri' 'Frankie' 'Aalopuri' 'Cold coffee'
 'Sandwich']

item_type
2
['Fastfood' 'Beverages']

item_price
5
[20 25 50 40 60]

quantity
15
[15  1  6  8 10  9 14  5  4  3 11 12  2 13  7]

transaction_amount
48
[300  20 150 200 225 280 100 400 160 180  60 220 600 250 440 520 260 500
 240  40 480 140 840 750 120  80  75 420 650 560 320 450 325 350 175 900
 550 780 360  25 660 125  50 375 275 540 720 700]

transaction_type
2
['Cash' 'Online']

received_by
2
['Mr.' 'Mrs.']

time_of_sale
5
['Afternoon' 'Night' 'Evening' 'Morning' 'Midnight']



In [172]:
df['transaction_type'] =  (df['transaction_type'] == 'Cash').astype(int)
df.head()

,item_name,item_type,item_price,quantity,transaction_amount,transaction_type,received_by,time_of_sale
1,Vadapav,Fastfood,20,15,300,1,Mr.,Afternoon
2,Vadapav,Fastfood,20,1,20,1,Mr.,Afternoon
3,Sugarcane juice,Beverages,25,6,150,0,Mr.,Night
4,Sugarcane juice,Beverages,25,8,200,0,Mr.,Evening
5,Vadapav,Fastfood,20,10,200,1,Mr.,Evening


In [173]:
x_train,x_test = train_test_split(df,test_size=0.2,random_state=42)
x_train,x_val = train_test_split(x_train,test_size=0.25, random_state=42)

x_train = x_train.reset_index(drop=True)
x_val = x_val.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)

y_train = x_train['transaction_type'].values
y_val = x_val['transaction_type'].values
y_test = x_test['transaction_type'].values

del x_train['transaction_type']
del x_val['transaction_type']
del x_test['transaction_type']

In [174]:
df.dtypes

item_name             object
item_type             object
item_price             int64
quantity               int64
transaction_amount     int64
transaction_type       int32
received_by           object
time_of_sale          object
dtype: object

In [175]:
categorical = ['item_name', 'item_type', 'received_by', 'time_of_sale']
numerical = ['item_price', 'quantity', 'transaction_amount']

In [176]:
def mi(series):
    return mutual_info_score(series,y_train)

In [177]:
x_train[categorical].apply(mi)

item_name       0.003705
item_type       0.000014
received_by     0.000030
time_of_sale    0.002236
dtype: float64

In [181]:
graph = x_train[numerical].corr()
graph

,item_price,quantity,transaction_amount
item_price,1.000000,-0.012697,0.612207
quantity,-0.012697,1.000000,0.707356
transaction_amount,0.612207,0.707356,1.000000


In [185]:
dv = DictVectorizer(sparse=False)

train_dicts = x_train.to_dict(orient = 'records')
x_train = dv.fit_transform(train_dicts)

val_dict = x_val.to_dict(orient='records')
x_val = dv.transform(val_dict)